In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

### Importing libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Dataset loading

In [ ]:
data = pd.read_csv("/kaggle/input/mnistlike-dataset-squarecircletriangle/train_data.csv")

print(f"data's shape: {data.shape}")
data.head()

### Visualisation

In [ ]:
from PIL import Image

fig = plt.figure(figsize=(5, 5))

rows = 3
columns = 3

for i in range(9):
    j=np.random.randint(0,200)
    fig.add_subplot(rows, columns, i+1)
    img_arr=data.iloc[j,1:].values.reshape(28,28)
    label=str(data.iloc[j,0])
    plt.imshow(img_arr)
    plt.axis('off')
    plt.title("Label"+' '+label)

### Data processing

In [ ]:
X=data.iloc[:,1:].values.astype('float32')
y=data.iloc[:,0].values.astype('int32')

In [ ]:
# Normalizing the data
X /=255.0

# Reshaping from 784 to 28,28,1 for inputing into Conv2D
X=X.reshape(-1,28,28,1)
print(f"new shape of X: {X.shape}")

# One-Hot-Encoding of the label
from keras.utils.np_utils import to_categorical
y = to_categorical(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=42)

### Generating the model

In [ ]:
from tensorflow import keras
'''
Conv2D will read the shape of 4D dimension (num, height, weight, channels)
MaxPooling2D will project it to lower dimension, to accerelate the process
Flatten will reduce the size to 1D array
Dense is a normal neuron point
Dropout will throw away all the unneccesary neuron

'''
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=64, kernel_size=7, input_shape=[28, 28, 1]),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding="SAME"),
    keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding="SAME"),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding="SAME"),
    keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding="SAME"),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(units=128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=3, activation='softmax'),
])

model.summary()

In [ ]:
model.compile(
    optimizer="rmsprop",
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',patience=2)

In [ ]:
history = model.fit(
    X_train,y_train,
    validation_data=(X_valid,y_valid),
    epochs=50,
    callbacks=[early_stop]
)

### Model Evaluation

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[0:, ['loss', 'val_loss']].plot()
plt.title("Loss")
plt.show()

In [ ]:
history_df.loc[0:, ['accuracy','val_accuracy']].plot()
plt.title("Accuracy")
plt.show()

In [ ]:
predictions = np.argmax(model.predict(X_valid), axis=-1)
to_categorical(predictions)

In [ ]:
labels=["circle","triangle","square"]
coded_labels=['[1. 0. 0.]','[0. 1. 0.]','[0. 0. 1.]']
pred_labels=[0,1,2]

arr_lab=dict(zip(coded_labels, labels))
int_lab=dict(zip(pred_labels, labels))

In [ ]:
f, axs = plt.subplots(2,2,figsize=(11,11))
for i in range (0,9):
    plt.subplot(3,3,i+1)
    plt.xticks([])
    plt.yticks([])
    img_nbr = np.random.randint(0,len(predictions))
    plt.imshow(X_valid[img_nbr])
    plt.title("True={}; Predicted={} ".format(
         arr_lab[str(y_valid[img_nbr])],
         int_lab[predictions[img_nbr]]
        )
    )

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_valid,to_categorical(predictions)))

### Submission

In [ ]:
test=pd.read_csv("/kaggle/input/mnistlike-dataset-squarecircletriangle/quiz_for_competition_r.csv")
test.head()
test.shape

In [ ]:
X_test=test.iloc[:,1:].values.astype('float32')

In [ ]:
# Normalizing the data
X_test /=255.0

# Reshaping from 784 to 28,28,1 for inputing into Conv2D
X_test=X_test.reshape(-1,28,28,1)

In [ ]:
predictions = np.argmax(model.predict(X_test), axis=-1)
predictions

In [ ]:
sub=pd.DataFrame({
    'Category': predictions
})
sub.index.name='ID'
sub

In [ ]:
sub.to_csv('submission1.csv')